In [1]:
import os, sys, json
sys.path.append("/workspace/AutoCog")
from autocog.language.sta.compile import Compiler
from autocog.utility.pynb import wrap_graphviz, display

In [2]:
sources = ["""
prompt mmlu_main {
    is {
        topic is text;
        question is text;
        choices[4] is {
            value is text;
            correct is enum("yes","no");
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return {
        from .answer;
    }
    annotate {
        .topic           as "the general category from which the question was taken";
        .question        as "the question that you have to answer";
        .choices         as "you judge whether each choice is correct or not";
        .choices.value   as "the value of the choice";
        .choices.correct as "you decide whether this choice is correct or not";
        .answer          as "you repeat the value of the choice that best answer the question";
    }
}""","""
format bool {
    is enum("true","false");
    annotate "a bolean value is either true or false";
}

struct a_choice {
    is {
        value   is text;
        correct is bool;
    }
    annotate {
        .value   as "the value of the choice";
        .correct as "you decide whether this choice is correct or not";
    }
}

prompt mmlu_main {
    is {
        topic      is text<20>;
        question   is text<length=50>;
        choices[4] is a_choice;
        answer     is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return .answer;
    annotate {
        .topic    as "the general category from which the question was taken";
        .question as "the question that you have to answer";
        .choices  as "you judge whether each choice is correct or not";
        .answer   as "you repeat the value of the choice that best answer the question";
    }
}
"""]

In [3]:
for source in sources:
    compiler = Compiler(source=source)
    for fld in compiler.program.prompts['mmlu_main'].fields:
        range = '' if fld.range is None else ( f'[{fld.range[0]}]' if fld.range[0] == fld.range[1] else f'[{fld.range[0]}:{fld.range[1]}]' )
        print(f"{'> '*fld.depth}{fld.name}({'record' if fld.format is None else fld.format.label()}){range}: {' '.join(fld.desc)}")
    print("--------------------")
    for (f,fmt) in compiler.program.formats.items():
        print(f"- {f}: {fmt}")
    print("--------------------")
    for (f,fld) in compiler.fields.items():
        print(f"- {f}: {fld}")
    print("====================")
    # wrap_graphviz(compiler.program.toGraphViz())

> topic(text): 
> question(text): 
> choices(record)[4]: 
> > value(text): 
> > correct(enum("yes","no")): 
> answer(repeat(choices.value)): 
--------------------
- mmlu_main.topic: name='mmlu_main.topic' desc=[] annonymous=True length=None within=None
- mmlu_main.question: name='mmlu_main.question' desc=[] annonymous=True length=None within=None
- mmlu_main.choices.value: name='mmlu_main.choices.value' desc=[] annonymous=True length=None within=None
- mmlu_main.choices.correct: name='mmlu_main.choices.correct' desc=[] annonymous=True values=['yes', 'no']
- mmlu_main.answer: name='mmlu_main.answer' desc=[] annonymous=True path=Path(steps=[('choices', None), ('value', None)], is_input=False, prompt=None) mode='repeat'
--------------------
- mmlu_main.topic: name='topic' desc=[] depth=1 format=Completion(name='mmlu_main.topic', desc=[], annonymous=True, length=None, within=None) range=None parent=Prompt(name='mmlu_main', desc=[], fields=[Field(name='topic', desc=[], depth=1, format=Compl